In [1]:
import cv2
import matplotlib.pyplot as plt
import math

import numpy as np

import torch
import torchvision
from torchvision import transforms, utils
import torch.nn as nn

import os
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

import csv

In [2]:
#!pip install selectivesearch

In [2]:
import selectivesearch

In [3]:
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

In [ ]:
ldir = os.listdir('./tiny-imagenet-200/train')
ldir = ldir[1:]

In [22]:
def create_dataset(dir_path, boxes_file_path):
    with open(boxes_file_path, mode="a") as w_file:
        file_writer = csv.writer(w_file, delimiter=",", lineterminator="\r")
        for root, dirs, files in os.walk(dir_path):
            for f in files:
                if f.endswith('.JPEG'):
                    im_name = os.path.join(root, f)
                    im = cv2.imread(im_name)
                    new_height = 200
                    new_width = int(im.shape[1]*200/im.shape[0])
                    new_im = cv2.resize(im, (new_width, new_height))

                    im_lbl, regions = selectivesearch.selective_search(new_im, scale=600, min_size=60)
                    candidates = set()
                    for r in regions:
                        if r['rect'] in candidates: continue
                        x, y, w, h = r['rect']
                        if r['size'] < 450: continue
                        if h == 0 or w == 0: continue
                        if w/h > 1.8 or h/w > 1.8: continue
                        candidates.add(r['rect'])

                    candidates_scaled = []
                    for x, y, w, h in candidates:
                        x_scaled = int(x*im.shape[1]/new_width)
                        y_scaled = int(y*im.shape[0]/new_height)
                        w_scaled = int(w*im.shape[1]/new_width)
                        h_scaled = int(h*im.shape[0]/new_height)
                        candidates_scaled.append(x_scaled)
                        candidates_scaled.append(y_scaled)
                        candidates_scaled.append(w_scaled)
                        candidates_scaled.append(h_scaled)

                    file_writer.writerow([im_name]+candidates_scaled)


In [23]:
bpath = './tiny-imagenet-200/train/boxes.csv'

In [ ]:
for d in ldir:
    cont_path = './tiny-imagenet-200/train/' + d
    create_dataset(cont_path, bpath)